### **RISCO DE FOGO: QUEIMADAS NO CERRADO**
<hr>
<p align="justify">
Este trabalho é referente as tarefas desenvolvidas no Bloco 1. Espera-se desenvolver um projeto capaz de utilizar os comandos aprendidos pela linguagem de programação Python, na disciplina de Aprendizado de Máquina, além de desenvolver um sistema de previsão através de Machine Learning. O grupo desenvolvedor é composto pelas discentes:
<p align="justify">
Isabela Bento Beneti </p>
<p align="justify">
Monyque Karoline de Paula Silva </p>
<p align="justify">
Sofia Baccega C.C. de Oliveira </p>
<p align="justify">
Sophia Figueiredo Michel </p>
<hr>

### **Início:**
<p align="justify">
Em um primento momento, iremos demonstrar o processo de tratamento dos dados obtidos via Database do INPE a respeito dos focos de incêndio ocorridos no Cerrado durante o período Janeiro-Julho(2022). É necessário realizar as padronizações de bibliotecas inicias, como a Pandas, Seaborn e Matplotlib, posteriormente, trabalha-se os dados até a otimização destes para a análise dos atríbutos escolhidos.
</p>

## **Coleta:**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from itertools import product
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

In [2]:
#Dados do Cerrado Brasileiro, armazenados mensalmente via .csv
arquivos = ['janeiro.csv','fev.csv','marco.csv','abril.csv','maio.csv','junho.csv','julho.csv']

In [3]:
def read_csv(list_of_files):
    dfs = []
    for x in list_of_files:
        file = pd.read_csv(x, sep=',')
        dfs.append(file)
    df = pd.concat(dfs).reset_index(drop=True)
        
    return df

In [4]:
meses = read_csv(arquivos)

In [5]:
#Dados trabalhados para a eliminação de NaN nas colunas e identificação dos tipos de dados
meses.convert_dtypes()
meses.dropna(how='any', inplace=True)

In [6]:
## Para rodar o código pela primeira vez, descomente essas linhas

meses.drop(columns=['Unnamed: 0','estado', 'municipio', 'pais', 'municipio_id', 'estado_id', 'pais_id', 'bioma'], axis = 1, inplace=True)

In [7]:
meses['satelite']= meses['data_hora_gmt']
meses['satelite'] = meses['satelite'].str[11:13] #criando coluna "hora" a partir de "data_hora_gmt"
meses['data_hora_gmt'] = meses['data_hora_gmt'].str[6] #criando coluna "mes" a partir de "data_hora_gmt"
meses_fogo = meses['risco_fogo'] > 0
meses.rename(columns={'satelite': 'hora'}, inplace= True)
meses.rename(columns={'data_hora_gmt': 'mes'}, inplace = True)
meses = meses[meses_fogo]
meses.describe

<bound method NDFrame.describe of               lat        lon mes hora  numero_dias_sem_chuva  precipitacao  \
775    -12.314859 -43.234066   3   01                    9.0           2.7   
778    -20.227880 -46.415940   3   01                    8.0           0.0   
779    -20.229250 -46.426820   3   01                    8.0           0.0   
780    -20.238930 -46.425510   3   01                    8.0           0.0   
781    -20.237550 -46.414600   3   01                    8.0           0.0   
...           ...        ...  ..  ...                    ...           ...   
281582  -6.360000 -45.670000   7   23                   46.0           0.0   
281583  -6.360000 -45.690000   7   23                   46.0           0.0   
281584  -6.400000 -44.140000   7   23                   34.0           0.0   
281585  -6.350000 -45.670000   7   23                   46.0           0.0   
281586  -6.340000 -45.690000   7   23                   45.0           0.0   

        risco_fogo  
775     

In [8]:
meses["mes"] = meses["mes"].astype(float)
meses["hora"] = meses["hora"].astype(float)
print(meses.dtypes)

lat                      float64
lon                      float64
mes                      float64
hora                     float64
numero_dias_sem_chuva    float64
precipitacao             float64
risco_fogo               float64
dtype: object


# Validação Cruzada

In [12]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "meses"
FEATURES = ["numero_dias_sem_chuva", "precipitacao", "lat", "lon","mes","hora"]
TARGET = ["risco_fogo"]



indices = meses.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = meses.loc[indices_treino]
df_teste = meses.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

In [13]:
# Cálculo do 𝑅²
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

NUM_ARVORES = 10
NUM_FOLDS = 10
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.95231729 0.95344754 0.94997776 0.95097113 0.94875552 0.94723071
 0.95004955 0.94973619 0.94836843 0.95024618]

A média dos scores é de:  0.9501100311365693


In [14]:
# Cálculo do RMSE com 10 árvores
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.05553981 -0.05536261 -0.0570356  -0.05740506 -0.05766428 -0.05837368
 -0.0572157  -0.05717934 -0.05759255 -0.05710838]

A média dos scores é de:  -0.057047700606306505


In [15]:
# Cálculo do RMSE com 100 árvores
NUM_ARVORES = 100
NUM_FOLDS = 10
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.05323635 -0.05233397 -0.05407626 -0.05366062 -0.05440683 -0.05549395
 -0.05440487 -0.0546266  -0.05442401 -0.05332203]

A média dos scores é de:  -0.053998550025994366


In [16]:
from sklearn.metrics import mean_squared_error

NUM_ARVORES = 100
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
modelo_rf.fit(X_treino, y_treino)

y_previsao = modelo_rf.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo floresta aleatória no conjunto de teste foi de {RMSE}")

O RMSE do modelo floresta aleatória no conjunto de teste foi de 0.05227392171070777 dólares.


## Pipelines

In [17]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

NUM_VIZINHOS = 3
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.08260132 -0.07909832 -0.08300158 -0.08282783 -0.08252275 -0.08178307
 -0.08253879 -0.08274307 -0.08203952 -0.08307247]

A média dos scores é de:  -0.082222873077635


In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

NUM_VIZINHOS = 3
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.08260132 -0.07909832 -0.08300158 -0.08282783 -0.08252275 -0.08178307
 -0.08253879 -0.08274307 -0.08203952 -0.08307247]

A média dos scores é de:  -0.082222873077635


In [19]:
NUM_VIZINHOS = 7

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)
modelo_knn_composto.fit(X_treino, y_treino)

y_previsao = modelo_knn_composto.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN no conjunto de teste foi de {RMSE}.")

O RMSE do modelo k-NN no conjunto de teste foi de 0.09231368591636432 dólares.
